In [ ]:
# Import dependencies
import pandas as pd
import regex as re
import json

In [ ]:
# Pull in life expectancy data
life_df = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/le_clean.csv")

In [ ]:
# Pull in gdp data
gdp = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/gdp_clean.csv")

In [ ]:
# Pull in social security and health system data
social = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/Cluster_Analysis_Data/country_social_security_systems-coded.csv")

In [ ]:
# Pull in region and subregion data
regions = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

In [ ]:
# Pull in per capita GDP data
per_cap = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/Vivek_Project/Clean_Data/world_bank_gdp_per_capita_clean.csv%20-%20API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4770417.csv")

In [ ]:
# Pull in alcohol and tobacco data
alc_tob = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/Vivek_Project/Clean_Data/alcohol_tobacco_normalized_clean.csv")

In [ ]:
alc_tob = alc_tob.rename({'2000 [YR2000': '2000 [YR2000]'}, axis=1)

In [ ]:
# Separate alcohol and tobacco data into separate dataframes
alc = alc_tob.loc[alc_tob['Series Name'] == 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)']
tob = alc_tob.loc[alc_tob['Series Name'] == 'Prevalence of current tobacco use (% of adults)']

In [ ]:
# Pull in health expenditure data
exp = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/Cluster_Analysis_Data/health_expenditure-clean.csv")

In [ ]:
# Pull in list of master country names and codes
master = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/master_country_list/country_profile_urls.csv")

# List of countries in master document
country_list = master['country'].to_list()

In [ ]:
# Add country code column to all csvs without country code column
for country in country_list:
  code_string = master.loc[master['country'] == country, 'country_code'].to_json()
  code_name = re.search('[A-Za-z]+', code_string).group(0)
  gdp.loc[gdp['Country Name'] == country, 'Country Code'] = code_name
  social.loc[social['country'] == country, 'Country Code'] = code_name
  alc.loc[alc_tob['Country Name'] == country, 'Country Code'] = code_name
  tob.loc[alc_tob['Country Name'] == country, 'Country Code'] = code_name
  exp.loc[exp['country'] == country, 'Country Code'] = code_name

In [ ]:
# Correct null value
tob.at[1, 'Country Code'] = 'ALB'

In [ ]:
# Load existing geojson point data
with open('countries (1).geojson') as f:
  data = json.load(f)

In [ ]:
# Add life expectancy and other metrics to geojson data
import re

for i in range(0,len(data['features'])):
  country_code = data['features'][i]['properties']['ISO_A3']
  for j in range(2000,2020):
    try:
      continent = regions.loc[regions['alpha-3'] == country_code, 'region'].to_json()
      continent = re.search(':"\D+', continent).group(0)
      continent = continent[2:-2]
      data['features'][i]['properties']['continent'] = continent
    except:
      data['features'][i]['properties']['continent'] = ''
    try:
      sub_region = regions.loc[regions['alpha-3'] == country_code, 'sub-region'].to_json()
      sub_region = re.search(':"\D+', sub_region).group(0)
      sub_region = sub_region[2:-2]
      data['features'][i]['properties']['sub-region'] = sub_region
    except:
      data['features'][i]['properties']['sub-region'] = ''   
    try:
      soc_string = social.loc[social['Country Code'] == country_code, 'system_type'].to_json()
      soc_string = re.search("\[.+\]", soc_string).group(0)
      social_sys = soc_string[1:-1]
      data['features'][i]['properties']['system'] = social_sys
    except:
      data['features'][i]['properties']['system'] = ''
    try:
      depth_string = social.loc[social['Country Code'] == country_code, 'sss_depth'].to_json()
      depth = depth_string[-2]
      data['features'][i]['properties']['system_depth'] = int(depth)
    except:
      data['features'][i]['properties']['system_depth'] = ''
    try:
      govt = exp.loc[social['Country Code'] == country_code, 'govt_he'].to_json()
      govt_exp_val = float(re.search('\d+\.\d+', govt).group(0))
      data['features'][i]['properties']['govt'] = round(float(govt_exp_val),2)
    except:
      data['features'][i]['properties']['govt'] = 0
    try:
      priv = exp.loc[social['Country Code'] == country_code, 'private_he'].to_json()
      priv_exp_val = float(re.search('\d+\.\d+', priv).group(0))
      data['features'][i]['properties']['priv'] = round(float(priv_exp_val),2)
    except:
      data['features'][i]['properties']['priv'] = 0    
    yr_string = str(j)
    try:
      expectancy = life_df.loc[life_df['Country Code'] == country_code, f'{yr_string}']
      expectancy = expectancy.to_json()
      expectancy = re.search("\d\d\.\d\d", expectancy).group(0)
      expectancy = float(expectancy)
      data['features'][i]['properties'][f'life_expectancy [YR{yr_string}]'] = round(float(expectancy),2)
    except:
      data['features'][i]['properties'][f'life_expectancy [YR{yr_string}]'] = 0
    try:
      gdp_string = gdp.loc[gdp['Country Code'] == country_code, f'{yr_string}'].to_json()
      gdp_val = float(re.search('\d+\.\d+', gdp_string).group(0))
      data['features'][i]['properties'][f'gdp [YR{yr_string}]'] = round(float(gdp_val),2)
    except:
      data['features'][i]['properties'][f'gdp [YR{yr_string}]'] = 0
    try:
      gdp_cap_string = per_cap.loc[per_cap['Country Code'] == country_code, f'{yr_string}'].to_json()
      gdp_cap_val = float(re.search('\d+\.\d+', gdp_cap_string).group(0))
      data['features'][i]['properties'][f'gdp_pcap [YR{yr_string}]'] = round(float(gdp_cap_val),2)
    except:
      data['features'][i]['properties'][f'gdp_pcap [YR{yr_string}]'] = 0
    try:
      alc_string = alc.loc[alc['Country Code'] == country_code, f'{yr_string} [YR{yr_string}]'].to_json()
      alc_level = float(re.search('\d+\.\d+', alc_string).group(0))
      data['features'][i]['properties'][f'alc [YR{yr_string}]'] = round(float(alc_level),2)
    except:
      data['features'][i]['properties'][f'alc [YR{yr_string}]'] = 0
    try:
      tob_string = tob.loc[tob['Country Code'] == country_code, f'{yr_string} [YR{yr_string}]'].to_json()
      tob_level = float(re.search('\d+\.\d+', tob_string).group(0))
      data['features'][i]['properties'][f'tob [YR{yr_string}]'] = round(float(tob_level),2)
    except:
      data['features'][i]['properties'][f'tob [YR{yr_string}]'] = 0

In [ ]:
# # # Write GeoJSON file with new data for tableau visualization
with open('float_file_round.geojson', 'w') as f:
   f.write(json.dumps(data))